In [2]:
#importacion de libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#carga de datasets
#Le especifico el tipo a algunas columnas porque sino me tira un warning, podria especificar el verdadero tipo tambien
todoSet = pd.read_csv("/home/jonathan/Escritorio/orgaDeDatos/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv",\
                   dtype = {3: object,5: object,
                    6: object,7: object,
                    8: object,9: object,
                    10: object,11: object,
                    12: object,13: object,
                    14: object,15: object,
                    16: object,17: object,
                    18: object,19: object,
                    20: object,21: object,
                    22: object})
usersJunioSet = pd.read_csv("/home/jonathan/Escritorio/orgaDeDatos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv")

In [4]:
usersJunioSet.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [12]:
#Merge
usersTest = todoSet[todoSet['person'].isin(usersJunioSet['person'])]

In [18]:
usersTest['timestamp'] = pd.to_datetime(usersTest['timestamp'])
#Creo una columna con solo fecha sin tiempo
usersTest['fecha'] = usersTest['timestamp'].dt.date

/home/jonathan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jonathan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [151]:
def agruparPorPersonaFecha(dataframe,evento,nuevoNombreEvento):
    feature = dataframe[dataframe['event'] == evento].sort_values(by = 'fecha')
    feature = feature.groupby(['fecha','person'])['event'].size()
    feature = pd.DataFrame(data = feature)\
                        .rename(columns = {'event' : nuevoNombreEvento})
    feature = feature.sort_values(by = ['fecha','person']).reset_index()
    feature['fecha'] = pd.to_datetime(feature['fecha'])
    feature['mes'] = feature['fecha'].dt.month
    feature = feature[['mes','person',nuevoNombreEvento]]
    feature = pd.DataFrame(data = feature.groupby(['mes','person'])[nuevoNombreEvento]\
                             .size()).unstack('mes',0).reset_index('person')
    array = feature[nuevoNombreEvento].iloc[:,:5].values
    dataset = pd.DataFrame({'Enero'+'_'+nuevoNombreEvento:array[:,-5],'Febrero'+'_'+nuevoNombreEvento:array[:,-4]\
                        ,'Marzo'+'_'+nuevoNombreEvento:array[:,-3],'Abril'+'_'+nuevoNombreEvento:array[:,-2],\
                        'Mayo'+'_'+nuevoNombreEvento:array[:,-1]})
    dataset['person'] = feature['person']

    dataset = change_column_order(dataset, 'person', 0)
    
    return dataset

In [172]:
# 1er feature: Cantidad de eventos por usuario
eventosPorUserTest = usersTest.groupby(['fecha','person'])['event'].size()
eventosPorUserTest = pd.DataFrame(data = eventosPorUserTest)\
                        .rename(columns = {'event' : 'cantEventos'})
eventosPorUserTest = eventosPorUserTest.sort_values(by = ['fecha','person']).reset_index()
eventosPorUserTest['fecha'] = pd.to_datetime(eventosPorUserTest['fecha'])
eventosPorUserTest['mes'] = eventosPorUserTest['fecha'].dt.month

eventosPorUserTest = eventosPorUserTest[['mes','person','cantEventos']]
eventosPorUserTest = pd.DataFrame(data = eventosPorUserTest.groupby(['mes','person'])['cantEventos']\
                             .size()).unstack('mes',0).reset_index('person')
array = eventosPorUserTest['cantEventos'].iloc[:,:5].values
dataset = pd.DataFrame({'Enero'+'_'+'cantEventos':array[:,-5],'Febrero'+'_'+'cantEventos':array[:,-4]\
                        ,'Marzo'+'_'+'cantEventos':array[:,-3],'Abril'+'_'+'cantEventos':array[:,-2],\
                        'Mayo'+'_'+'cantEventos':array[:,-1]})
dataset['person'] = eventosPorUserTest['person']

dataset = change_column_order(dataset, 'person', 0)
eventosPorUserTest = dataset
eventosPorUserTest.head()

,person,Enero_cantEventos,Febrero_cantEventos,Marzo_cantEventos,Abril_cantEventos,Mayo_cantEventos
0,00091926,0,0,0,0,22
1,00091a7a,0,0,1,0,0
2,000ba417,0,0,0,0,3
3,000e4d9e,0,0,0,0,9
4,000e619d,0,0,0,0,4


In [171]:
#2do feature: cantidad de checkouts por user por mes

usersChecks = agruparPorPersonaFecha(usersTest,'checkout','checkouts')

In [155]:
#3er feature: cantidad de leads por mes por user
usersLeads = agruparPorPersonaFecha(usersTest,'lead','leads')

In [157]:
#4to feature: cantidad de conversiones por mes por user
usersConversions = agruparPorPersonaFecha(usersTest,'conversion','conversiones')

In [158]:
#5to feature: cantidad de staticPages visitadas por dia por user
usersStaticPages = agruparPorPersonaFecha(usersTest,'staticpage','staticPageVisitadas')

In [159]:
#6to feature: cantidad de viewed product por mes por usuario
usersViewedProducts = agruparPorPersonaFecha(usersTest,'viewed product','ProductosVistosPorUser')

In [160]:
#7mo feature: cantidad de listados visitados por usuario por mes
usersBrandListing = agruparPorPersonaFecha(usersTest,'brand listing','listadosVisitados')

In [162]:
#9no feature: cantidad de searched products
usersSearchProduct = agruparPorPersonaFecha(usersTest,'searched products','productosBuscados')

In [163]:
#10mo feature: cantidad de veces que el usuario ingresa al sitio mediante campania de marketing online
usersCampaignMarketing = agruparPorPersonaFecha(usersTest,'ad campaign hit','accesos por marketing online')

In [164]:
#11 feature: cantidad de veces que el user ingresa al sitio mediante search engine
usersSearchEngine =  agruparPorPersonaFecha(usersTest,'search engine hit','searchEngine')


In [165]:
#12 feature: cantidad de veces que el usuario visita la homepage
usersHomePage = agruparPorPersonaFecha(usersTest,'generic listing','accesos a la homepage')

#### Armado de la matriz de features numericos
Entiendo tiene mas sentido tomar peridos mensuales en lugar de diariamente entonces tendria n columnas de features numericos multiplicado por m donde m=cantidad de periodos tomados en este caso m=5

In [174]:
#Hago el join 
join1 = pd.merge(eventosPorUserTest,usersChecks, on='person',how='left')
join2 = pd.merge(usersLeads, usersConversions, on='person',how='left')
join3 = pd.merge(usersStaticPages, usersViewedProducts, on='person',how='left')
join5 = pd.merge(usersSearchProduct, usersCampaignMarketing, on='person',how='left')
join6 = pd.merge(usersSearchEngine, usersHomePage, on='person',how='left')

join12 = pd.merge(join1,join2, on='person',how='left')
join34 = pd.merge(join3,join5, on='person',how='left')
join56 = pd.merge(join6,usersBrandListing, on='person',how='left')

join12n34 = pd.merge(join12,join34, on='person',how='left')
datosNumericosTest = pd.merge(join12n34,join56, on='person',how='left').fillna(0)


In [177]:
datosNumericosTest.to_csv('matrizDatosNumericos.csv')